In [42]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from constantes import WINSIG_SERVICIO_FARMACIA_CC_SIGCOM

In [43]:
def desglosar_centro_de_costo(desglose, total_dinero):
    con_dinero = desglose.copy()
    con_dinero['TOTAL_X_PORCENTAJE'] = con_dinero['PORCENTAJES'] * total_dinero

    return con_dinero


In [44]:
df_winsig = pd.read_excel('input\\Informe Winsig Septiembre 2022.xlsx', header=7)
df_winsig = df_winsig.dropna(axis=1, how='all').dropna(axis=0, how='all')
archivo_produccion = pd.ExcelFile('input\\output_producciones.xlsx')


In [45]:
desglose_pabellon = pd.read_excel(archivo_produccion, sheet_name='PABELLÓN')
total_pabellon = df_winsig.query('SERVICIO == "B. PABELLON"')['Gasto Servicio'].iloc[0]
con_gastos_pabellon = desglosar_centro_de_costo(desglose_pabellon, total_pabellon)

In [46]:
desglose_policlinico = pd.read_excel(archivo_produccion, sheet_name='CONSULTAS SIN MANEJO DEL DOLOR')
total_policlinico = df_winsig.query('SERVICIO == "POLICLÍNICO"')['Gasto Servicio'].iloc[0]
con_gastos_policlinico = desglosar_centro_de_costo(desglose_policlinico, total_policlinico)


In [47]:
con_gastos_policlinico

,EGRESOS,ENERO,PORCENTAJES,AGRUPACION,TOTAL_X_PORCENTAJE
0,CONSULTA CARDIOLOGIA,324,0.09,CONSULTAS SIN MANEJO DEL DOLOR,"32,424,464.96"
1,CONSULTA CIRUGIA CARDIACA,302,0.08,CONSULTAS SIN MANEJO DEL DOLOR,"30,222,803.76"
2,CONSULTA CIRUGIA GENERAL (cirugía torax),292,0.08,CONSULTAS SIN MANEJO DEL DOLOR,"29,222,048.67"
3,CONSULTA NEUMOLOGIA (broncopulmonar),1694,0.46,CONSULTAS SIN MANEJO DEL DOLOR,"169,527,912.50"
4,CONSULTA NUTRICION,9,0.00,CONSULTAS SIN MANEJO DEL DOLOR,"900,679.58"
5,CONSULTA ONCOLOGIA,183,0.05,CONSULTAS SIN MANEJO DEL DOLOR,"18,313,818.17"
6,CONSULTA OTROS PROFESIONALES,857,0.23,CONSULTAS SIN MANEJO DEL DOLOR,"85,764,711.34"
7,CONSULTAS SIN MANEJO DEL DOLOR,3661,1.00,CONSULTAS SIN MANEJO DEL DOLOR,"366,376,439.00"
